In [ ]:
# GPU: 32*40 in 8.00s = 160/s
# CPU: 32*8 in 115.0s = 2/s

In [1]:
using Pkg; installed = Pkg.installed()
for p in ("Knet", "KnetLayers", "Images", "BSON")
    haskey(installed,p) || Pkg.add(p)
end
include("./resnetlib.jl")
gpu()

0

In [2]:
println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.installed()["Knet"])
println("Cores: ", chop(read(pipeline(`cat /proc/cpuinfo`,`grep processor`,`wc -l`),String)))
print("GPUs:\n", read(`nvidia-smi --query-gpu=gpu_name --format=csv,noheader`,String))

OS: Linux
Julia: 1.0.3
Knet: 1.1.1
Cores: 6
GPUs:
Tesla K80


In [3]:
const BATCH_SIZE = 32
const RESNET_FEATURES = 2048
const BATCHES_GPU = 40
const BATCHES_CPU = 8

8

In [4]:
# Create batches of fake data
function fakedata(batches; atype=KnetArray)
    x = rand(Float32, 224, 224, 3, BATCH_SIZE * batches)
    minibatch(x, BATCH_SIZE, xtype=atype)
end

fakedata (generic function with 1 method)

In [5]:
# Return features from classifier
function predictfn(network, data)
    out = []
    for x in data
        pred = network.layers[1:19](x)
        push!(out, mat(pred))
    end
    return Array(hcat(out...))
end

predictfn (generic function with 1 method)

## 1. GPU

In [6]:
# Initialize resnet weights and fake data
network = nothing; Knet.gc() # clear memory from previous run
KnetLayers.settype!(KnetArray{Float32})
network = trained(ResNet50);

In [7]:
@info("Cold start")
gpudata1 = fakedata(BATCHES_GPU, atype=KnetArray)
@time predictfn(network, gpudata1);

┌ Info: Cold start
└ @ Main In[7]:1


 26.287662 seconds (11.70 M allocations: 1.298 GiB, 36.44% gc time)


In [8]:
@info("Benchmarking")
gpudata = fakedata(BATCHES_GPU, atype=KnetArray)
@time predictfn(network, gpudata);

┌ Info: Benchmarking
└ @ Main In[8]:1


 10.889850 seconds (588.71 k allocations: 766.245 MiB, 25.19% gc time)


## 2. CPU

In [ ]:
# Initialize resnet weights and fake data
network = nothing; Knet.gc() # clear memory from previous run
KnetLayers.settype!(Array{Float32})
network = trained(ResNet50);

In [ ]:
@info("Cold start")
cpudata1 = fakedata(1, atype=Array);
@time predictfn(network, cpudata1);

In [ ]:
@info("Benchmarking")
cpudata = fakedata(BATCHES_CPU, atype=Array);
@time predictfn(network, cpudata);